In [1]:
import pandas as pd
import re
import pickle
import os.path
import datetime as dt
import pandas as pd
import numpy as np
import sys
#sys.path.append(os.path.abspath('/Users/utkarsh.lal/Desktop/forecasting_azure/forecasting/app/data'))
#print(sys.path)
import config as cfg
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX

# from data.data_management import create_directory, save_dict_as_pickle, load_dict_from_pickle
# from data.data_utils import DATA_LOADER, get_level_wise_contribution

In [2]:
#curr = os.getcwd()

In [3]:
# from pathlib import Path
# def get_project_root() -> Path:
#     os.path.join(os.path.dirname(curr))
#     return Path(curr).parent

In [4]:
# ROOT = get_project_root()
# ROOT

In [5]:
df_pd = pd.read_csv('/Users/utkarsh.lal/Desktop/forecasting_Beauty_clickstream/forecasting/TIRA - Beauty Clickstream Data.csv')
df_pd

,Product Category Level 1,Product Category Level 2,Product Category Level 3,Product List Clicks,Product Adds To Cart,Unique Purchases
0,Beauty,Make-Up,Lips,"1,31,783","25,817","7,177"
1,Beauty,Make-Up,Nails,"1,06,918","27,672","8,920"
2,Beauty,Make-Up,Face Makeup,"68,249","15,812","4,969"
3,Beauty,Make-Up,Eyes,"56,337","14,957","4,725"
4,Beauty,Hair,Hair Care,"47,005","10,300","4,059"
...,...,...,...,...,...,...
276,Beauty,Men's Grooming,VEGA Men T1 Beard Trimmer For Men With 40 Mins...,0,3,2
277,Beauty,Men's Grooming,VEGA Men X3 Beard Trimmer For Men With Quick C...,0,1,1
278,Beauty,Men's Grooming,Volamena Beard Growth Oil - Red Onion & Lavend...,0,1,0
279,Beauty,Moisturizers,Face Moisturizers,0,2,0


In [6]:
df_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281 entries, 0 to 280
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Product Category Level 1  281 non-null    object
 1   Product Category Level 2  281 non-null    object
 2   Product Category Level 3  281 non-null    object
 3   Product List Clicks       281 non-null    object
 4   Product Adds To Cart      281 non-null    object
 5   Unique Purchases          281 non-null    object
dtypes: object(6)
memory usage: 13.3+ KB


In [7]:
list_cols = ['Product List Clicks', 'Product Adds To Cart', 'Unique Purchases']
for cols in list_cols:
    df_pd[cols] = df_pd[cols].str.replace(',', '')
    df_pd[cols] = df_pd[cols].astype(int)
df_pd

,Product Category Level 1,Product Category Level 2,Product Category Level 3,Product List Clicks,Product Adds To Cart,Unique Purchases
0,Beauty,Make-Up,Lips,131783,25817,7177
1,Beauty,Make-Up,Nails,106918,27672,8920
2,Beauty,Make-Up,Face Makeup,68249,15812,4969
3,Beauty,Make-Up,Eyes,56337,14957,4725
4,Beauty,Hair,Hair Care,47005,10300,4059
...,...,...,...,...,...,...
276,Beauty,Men's Grooming,VEGA Men T1 Beard Trimmer For Men With 40 Mins...,0,3,2
277,Beauty,Men's Grooming,VEGA Men X3 Beard Trimmer For Men With Quick C...,0,1,1
278,Beauty,Men's Grooming,Volamena Beard Growth Oil - Red Onion & Lavend...,0,1,0
279,Beauty,Moisturizers,Face Moisturizers,0,2,0


In [8]:
df_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281 entries, 0 to 280
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Product Category Level 1  281 non-null    object
 1   Product Category Level 2  281 non-null    object
 2   Product Category Level 3  281 non-null    object
 3   Product List Clicks       281 non-null    int64 
 4   Product Adds To Cart      281 non-null    int64 
 5   Unique Purchases          281 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 13.3+ KB


In [9]:
pd.set_option('display.max_rows', None)

In [10]:
w_clicks=0.2
w_cart=0.3
w_uniq = 0.5

In [11]:
df_pd['popularity'] = np.round(
            ((w_clicks * df_pd['Product List Clicks'] + w_cart * df_pd['Product Adds To Cart'] + w_uniq * df_pd['Unique Purchases']) / (w_clicks + w_cart + w_uniq)) * 0.001, 5)

In [12]:
df_pd

,Product Category Level 1,Product Category Level 2,Product Category Level 3,Product List Clicks,Product Adds To Cart,Unique Purchases,popularity
0,Beauty,Make-Up,Lips,131783,25817,7177,37.6902
1,Beauty,Make-Up,Nails,106918,27672,8920,34.1452
2,Beauty,Make-Up,Face Makeup,68249,15812,4969,20.8779
3,Beauty,Make-Up,Eyes,56337,14957,4725,18.1170
4,Beauty,Hair,Hair Care,47005,10300,4059,14.5205
5,Beauty,Personal Care,Face Personal Care,37557,8262,3129,11.5545
6,Beauty,Skin Care,Masks,24698,5176,1561,7.2729
7,Beauty,Personal Care,Bath & Shower,22037,4437,1639,6.5580
8,Beauty,Fragrances,Unisex,21383,4848,1535,6.4985
9,Beauty,Skin Care,Toners & Serums,19274,2787,801,5.0914


In [ ]:
df_catalog = pd.read_parquet('/Users/utkarsh.lal/Desktop/forecasting_Beauty_clickstream/forecasting/datasets/cleaned_data_2020-10-01_2021-10-18.parquet')
df_catalog